<a href="https://colab.research.google.com/github/jjordana/twitter_sentiment_analysis/blob/master/roBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kaggle's API

In [0]:
!pip install -q kaggle
!mkdir .kaggle

In [0]:
import json
token = {"username":"jjordana16","key":"6e806145f7c3fdd4c09e7299f3a70d73"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [5]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v/content

- path is now set to: /content


In [6]:
!kaggle competitions download -c tweet-sentiment-extraction

  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 68.5MB/s]
  0% 0.00/41.4k [00:00<?, ?B/s]
100% 41.4k/41.4k [00:00<00:00, 35.8MB/s]
  0% 0.00/307k [00:00<?, ?B/s]
100% 307k/307k [00:00<00:00, 95.4MB/s]


## Importing Libraries & data

In [7]:
import pandas as pd, numpy as np
import tensorflow as tf
print('TF version',tf.__version__)
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold

TF version 2.2.0-rc4


In [0]:
pip install transformers

In [0]:
from transformers import *
import tokenizers
from tokenizers import ByteLevelBPETokenizer

In [0]:
train = pd.read_csv('/content/competitions/tweet-sentiment-extraction/train.csv.zip').fillna('')
train = train.dropna()

In [0]:
tokenizer = ByteLevelBPETokenizer(
    add_prefix_space=True, 
    lowercase=True,
    vocab_file='vocab-roberta-vocab.json', 
    merges_file='vocab-roberta-merges.txt')

In [0]:
tokenizer.train(["/content/train.csv"], vocab_size=20000)

In [38]:
tokenizer.save(directory="/content/", name='vocab-roberta')

['/content/vocab-roberta-vocab.json', '/content/vocab-roberta-merges.txt']

In [40]:
MAX_LEN = 96
EPOCHS = 3
BATCH_SIZE = 32
PAD_ID = 1
SEED = 777
LABEL_SMOOTHING = 0.1
tf.random.set_seed(SEED)
np.random.seed(SEED)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [0]:
rowsFile = train.shape[0]
input_ids = np.ones((rowsFile,MAX_LEN),dtype='int32')
attention_mask = np.zeros((rowsFile,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((rowsFile,MAX_LEN),dtype='int32')
start_tokens = np.zeros((rowsFile,MAX_LEN),dtype='int32')
end_tokens = np.zeros((rowsFile,MAX_LEN),dtype='int32')

We have already our tokenized our data.

1. Firstly, we find the index in which our `selected_text` starts in our `text` <br>(i.e. if out tweet is _"I love singning under the rain"_ and our selected_text _love singing_, we will find the index where it starts.<br>
It will be `3`).<br>
2. In an empty vector, `chars`, full of 0s, we fulfill it with ones where the text and selected_text `coincide`. <br>
3. We **encode** our tweet using the __tokenicer__. <br>
4. We get the whole length for each tweet

In [0]:
for val in range(rowsFile):
  # FIND OVERLAP
  text1 = " " + " ".join(train.loc[val,'text'].split())
  text2 = " ".join(train.loc[val,'selected_text'].split())

  idx = text1.find(text2) # We find index where our selected_word starts
  chars = np.zeros((len(text1))) # We create a vector of 0s with the length of text
  chars[idx:idx+len(text2)] = 1 # Fullfill the vector with 1 when it coincides
  if text1[idx-1]==' ': chars[idx-1] = 1 
  enc = tokenizer.encode(text1) 

  # ID_OFFSETS
  offsets = []
  idx=0
  for t in enc.ids:
    w = tokenizer.decode([t])
    offsets.append((idx,idx+len(w)))
    idx += len(w)

  # START END TOKENS
  toks = []
  for i,(a,b) in enumerate(offsets):  
    sm = np.sum(chars[a:b])
    if sm>0: toks.append(i) 
        
  s_tok = sentiment_id[train.loc[val,'sentiment']]
  input_ids[val,:len(enc.ids) + 5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
  attention_mask[val,:len(enc.ids)+5] = 1
  if len(toks)>0:
    start_tokens[val,toks[0]+1] = 1
    end_tokens[val,toks[-1]+1] = 1

TEST DATA

In [0]:
test = pd.read_csv('/content/competitions/tweet-sentiment-extraction/test.csv').fillna('')
rowsFile = test.shape[0]
input_ids_t = np.ones((rowsFile,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((rowsFile,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((rowsFile,MAX_LEN),dtype='int32')


In [0]:
for k in range(rowsFile):   
  # INPUT_IDS
  text1 = " "+" ".join(test.loc[k,'text'].split())
  enc = tokenizer.encode(text1)                
  s_tok = sentiment_id[test.loc[k,'sentiment']]
  input_ids_t[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
  attention_mask_t[k,:len(enc.ids)+5] = 1

In [0]:
import requests
r = requests.get('https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-config.json')
data = r.json()

In [0]:
import json
import requests

solditems = requests.get('https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-config.json') # (your url)
data = solditems.json()
with open('data.json', 'w') as f:
    json.dump(data, f)